# 🤖 BERT Question Answering with Transformers

This notebook demonstrates how a pre-trained BERT model performs
**extractive question answering** by predicting the start and end
positions of an answer within a given context.


## 1️⃣ What is BERT?

BERT (Bidirectional Encoder Representations from Transformers) is a
Transformer-based model trained on large text corpora.

Key points:
- Trained on the **BooksCorpus** dataset (~11,000 books)
- Trained on a large **Wikipedia** text corpus
- Uses **bidirectional context**, reading text left-to-right and right-to-left
- BERT Base has ~110M parameters
- BERT Large has ~340M parameters


In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

## 2️⃣ Loading a Pre-trained BERT Model

Here we load a BERT model fine-tuned for **question answering** on the
SQuAD dataset, along with its corresponding tokenizer.


In [ ]:
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'

In [ ]:
model =  BertForQuestionAnswering.from_pretrained(model_name)   #loading the pre-trained model

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)   #loading the pre-trained tokenizer

## 3️⃣ Preparing Question and Context

To perform question answering, BERT requires:
- A question
- A context passage containing the answer

Both are encoded together as a single input sequence.


In [ ]:
#example question and text containing the answer
question = "When was the first dvd released?"


In [63]:
answer_document = "The first DVD (Digital Versatile Disc) was released on March 24, 1997. It was a movie titled 'Twister' and was released in Japan. DVDs quickly gained popularity as a replacement for VHS tapes and became a common format for storing and distributing digital video and data."

## 4️⃣ Tokenizing Question–Context Pairs

The tokenizer encodes the question and context together, producing:
- Input IDs
- Token type (segment) IDs
- Special tokens such as `[CLS]` and `[SEP]`


In [ ]:
encoding = bert_tokenizer.encode_plus(text= question, text_pair=answer_document)  #tokenizing the question and answer text pairs 

In [ ]:
print(encoding)

In [ ]:
inputs = encoding['input_ids']  #getting the input ids from the encoding
sentence_embeddings = encoding['token_type_ids']   #getting the segment ids from the encoding
tokens =  tokenizer.convert_ids_to_tokens(inputs)   #converting the input ids to tokens

## 5️⃣ Running the BERT Model

The encoded inputs are passed into the BERT question answering model.
The model outputs:
- Start logits (where the answer begins)
- End logits (where the answer ends)


In [ ]:
tokenizer.decode(101)

In [ ]:
tokenizer.decode(102)

## 5️⃣ Running the BERT Model

The encoded inputs are passed into the BERT question answering model.
The model outputs:
- Start logits (where the answer begins)
- End logits (where the answer ends)


In [ ]:
output =  model(input_ids = torch.tensor([inputs]), token_type_ids= torch.tensor([sentence_embeddings]) )  #getting the model output 

## 6️⃣ Extracting the Answer Span

The answer is extracted by selecting the tokens with the highest
start and end scores and joining them into a readable string.


In [ ]:
start_index = torch.argmax(output.start_logits)
end_index = torch.argmax(output.end_logits)

print(start_index)
print(end_index)

In [ ]:
answer = ' '.join(tokens[start_index: end_index+1])  #getting the answer from the tokens
print(answer)

## 7️⃣ Visualizing Start and End Scores

To understand model confidence, we visualize:
- Start logits per token
- End logits per token

Higher values indicate stronger confidence that a token
marks the beginning or end of the answer.


In [ ]:
import matplotlib as plt 
import seaborn as sns


In [ ]:
s_scores = output.start_logits.detach().numpy().flatten()
e_scores = output.end_logits.detach().numpy().flatten()

In [ ]:
tokens_labels = []   #we want token labels as a list of strings with token and its index
for (i, token) in enumerate(tokens):
    tokens_labels.append('{:} - {:>2}'.format(token, i))

In [ ]:
ax =  sns.barplot(x=tokens_labels, y=s_scores) 
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")
ax.grid(True)

In [ ]:
ax =  sns.barplot(x=tokens_labels, y=e_scores)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")
ax.grid(True)

## ✅ Key Takeaways

- BERT performs extractive question answering using token span prediction
- Question and context are processed together
- Start and end logits determine the answer
- Visualization helps interpret model behavior
